# 국민재난안전포털 웹 크롤러 만들기

In [7]:
from requests import Session
from requests.compat import urljoin
from bs4 import BeautifulSoup

In [59]:
session = Session()

base_url = 'http://www.safekorea.go.kr/'
main_url = 'idsiSFK/neo/ext/json/main/menuList.json'

main_resp = session.get(urljoin(base_url, main_url))
result = main_resp.json()

disaster_service_url = ''
disaster_menu_seq = 1

# 재난문자 url과 params인 munuSeq 구하기
for cnt in range(0,len(result)):
    if cnt > 0 and result[cnt]['MENU_NM'] == '재난문자':
        disaster_service_url = 'idsiSFK' + result[cnt]['SERVICE_URL']
        disaster_menu_seq = result[cnt]['MENU_SEQ']
        break
        
#disaster_service_url, disaster_menu_seq

## 아래 코드 사용 불가
- post 방식의 재난문자 json request시 500 error 발생

In [75]:
session.cookies.clear()
disaster_resp = session.request('GET', urljoin(base_url, disaster_service_url), params={'menuSeq':disaster_menu_seq})
#dom = BeautifulSoup(disaster_resp.text, 'html.parser')
referer = disaster_resp.request.url

#새 쿠키값 
cookiedict = {}
for _ in session.cookies.items():
    cookiedict[_[0]] = _[1]

headers = {
    'Referer' : referer,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

#쿠키 날짜 value는 당일 날짜로 변경해야함, 실시간 크롤링시
cookies = {
    'JSESSIONID':cookiedict['JSESSIONID'],
    'elevisor_for_j2ee_uid':cookiedict['elevisor_for_j2ee_uid'],
    'e4d3f82b588e6cf132ee4765403cd800':cookiedict['e4d3f82b588e6cf132ee4765403cd800'],
    '3053195ebb01870828aed202f55dc974':'76ff93199833d41c33e1b89a9862350b',
    'popup_1':'20200802',
    'popup_2':'20200802',
    '_ga':'GA1.3.522710010.1595930482',
    '_gid':'GA1.3.1758721736.1596362546'
}

#session.request지만 누락된 쿠키값이 있어서 따로 다시 입력
disaster_list = 'idsiSFK/bbs/user/selectBbsList.do'
disaster_list_resp = session.request('POST', urljoin(base_url, disaster_list), headers=headers, cookies=cookies)

In [76]:
disaster_list_resp.request, disaster_list_resp.reason, disaster_list_resp.status_code, disaster_list_resp.headers

(<PreparedRequest [POST]>,
 'Internal Server Error',
 500,
 {'Content-Encoding': 'gzip', 'Content-Type': 'text/html;charset=ISO-8859-1', 'Content-Length': '254', 'Date': 'Sun, 02 Aug 2020 14:56:23 GMT'})

# 셀레니움 활용 실시간 문자 수집
- 셀레니움 사용 이유: 재난문자 페이지의 json request 불가(inter server error, 500 error)

In [77]:
from selenium import webdriver

In [83]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

uu = urljoin(base_url, disaster_service_url)
uuu = uu + '&menuSeq=' + str(disaster_menu_seq)
driver.get(uuu)

#### 재난문자 첫 페이지 => 10개 재난문자 가져오기
- 재난문자 하나씩 접근하므로 중간에 재난문자 갱신이 되면 똑같은 문자 접근할 수 있음

In [104]:
for cnt in range(0, 10):
    # cnt 번째 페이지, 첫 번째 재난 문자 path
    disaster_path = '//tbody[@id="bbs_tr"]/tr[@id="bbs_tr_{0}_apiData1"]/td[2]'.format(cnt)
    result_topic = driver.find_element_by_xpath(disaster_path)
    print((cnt+1), '번째 재난문자:', result_topic.text)

    # cnt 번째 재난문자 링크 이동
    first_a = disaster_path + '/a'
    last_url = driver.current_url
    driver.find_element_by_xpath(first_a).click()
    driver.implicitly_wait(3)

    # 재난문자 상세내용 확인
    disaster_context = '//div[@id="content"]/div[@id="content"]/div/div[@class="boardView"]/div[@class="boardView_context"]'
    result_context = driver.find_element_by_xpath(disaster_context)
    print(result_context.text)
    print('\n')
    driver.implicitly_wait(3)

    # 이전 페이지로 돌아가기
    driver.get(last_url)

1 번째 재난문자: 2020/08/03 13:02:21 재난문자[행정안전부]
[행정안전부] 오늘 13시00분 세종,충남(보령,청양,공주) 호우경보, 산사태ㆍ상습침수 등 위험지역 대피, 외출자제 등 안전에 주의바랍니다

-송출지역-
충청남도 공주시
충청남도 보령시
충청남도 청양군
세종특별자치시


2 번째 재난문자: 2020/08/03 13:00:44 재난문자[천안시청]
[천안시청] 쌍용지하차도 침수에 따라 통제 예정이니, 우회 통행하여 주시기 바랍니다.

-송출지역-
충청남도 천안시


3 번째 재난문자: 2020/08/03 12:58:31 재난문자[강남구재대본]
[강남구재대본]팔당댐 방류에 따른 한강수위 상승으로 강남, 신청담, 신사나들목, 신사수문, 압구1·수문 14시 폐쇄 예정이오니, 차량진입이 통제됨을 알려드립니다.

-송출지역-
서울특별시 강남구


4 번째 재난문자: 2020/08/03 12:55:21 재난문자[한강홍수통제소]
[한강홍수통제소]오늘 12:50 홍천강 홍천군(홍천교) 홍수주의보 발령, 안전에 유의하시기 바랍니다.

-송출지역-
강원도 춘천시
강원도 홍천군
경기도 가평군


5 번째 재난문자: 2020/08/03 12:40:39 재난문자[오산시청]
[오산시청] 오늘 많은 비로 오산천 수위 상승중, 오산천 산책로 및 자전거도로 통행을 금지합니다.[재난안전대책본부]

-송출지역-
경기도 오산시


6 번째 재난문자: 2020/08/03 12:32:41 재난문자[천안시청]
[천안시청] 청수지하차도, 청당지하차도, 남산지하도, 신방동 하상도로 통제되었으니 우회 통행하여 주시기 바랍니다.

-송출지역-
충청남도 천안시


7 번째 재난문자: 2020/08/03 12:25:49 재난문자[용인시청]
[용인시청] 용인시 호우경보 발효중. 집중호우가 예상되오니 산사태·상습침수 등 위험지역에서 대피하시고 차량운행 및 외출 자제 등 안전에 주의바랍니다.

-송출지역-
경기도 용인시


8 번째 재난문자: 2020/08/03 12:25:47 재난문자[광진구청